In [55]:
from qiskit import QuantumCircuit, execute, Aer, QuantumRegister, ClassicalRegister
import numpy as np

def print_state(dense_state):
    """
    Prints a *dense_state* with kets. Compatible with quimb states.

    Parameters
    ----------
    dense_state: array_like
            Dense representation of a quantum state

    Returns
    -------
    None: None
    """

    NN = int(np.log2(len(dense_state)))

    binaries = [bin(ii)[2:] for ii in range(2**NN)]
    binaries = ['0'*(NN-len(a)) + a for a in binaries] #Pad with 0s

    ket = []
    for ii, coef in enumerate(dense_state):
        if not np.isclose(np.abs(coef), 0.):
            if np.isclose(np.imag(coef), 0.):
                if np.isclose(np.real(coef), 1.):
                    ket.append('|{}>'.format(binaries[ii]))
                else:
                    ket.append('{:.3f}|{}>'.format(np.real(coef), binaries[ii]))
            else:
                ket.append('{:.3f}|{}>'.format(coef, binaries[ii]))
    print(' + '.join(ket))

def qiskit_get_statevect(qc):
    """
        Returns the statevector of the qiskit quantum circuit *qc*

        Parameters
        ----------
        qc: Quantum circuit
            Quantum circuit of which we want the statevector

        Returns
        -------
        st: array_like
            Statevector of the quantum circuit after the application
            of the reverse operation on the qubit's ordering
    """
    statevector = execute(qc, Aer.get_backend('statevector_simulator')).result()
    statevector = statevector.get_statevector()

    return statevector

We first create a bell state with all the 5 qubits. Notice that we have a connectivity map with a X

In [60]:
qreg = QuantumRegister(5)
creg = ClassicalRegister(1)
qc = QuantumCircuit( qreg, creg)

qc.h(2)
for ii in range(5):
    if ii==2:
        continue
    qc.cx(2, ii)

qc.h(2)
qc.cz(0, 2)
qc.measure(2, 0)

print(qc)

             ┌───┐                          
q180_0: ─────┤ X ├─────────────────────■────
             └─┬─┘┌───┐                │    
q180_1: ───────┼──┤ X ├────────────────┼────
        ┌───┐  │  └─┬─┘          ┌───┐ │ ┌─┐
q180_2: ┤ H ├──■────■────■────■──┤ H ├─■─┤M├
        └───┘          ┌─┴─┐  │  └───┘   └╥┘
q180_3: ───────────────┤ X ├──┼───────────╫─
                       └───┘┌─┴─┐         ║ 
q180_4: ────────────────────┤ X ├─────────╫─
                            └───┘         ║ 
  c1: 1/══════════════════════════════════╩═
                                          0 


In [61]:
statevect = qiskit_get_statevect(qc)
print_state(statevect)

0.707|00100> + 0.707|11111>


In [62]:
qc1 = qc.copy()
qc1.x(0)
qc1.reset(2)
qc1.cx(0, 2)
qc1.cx(1, 2)
qc1.cx(3, 2)
qc1.cx(4, 2)
qc1.measure(2, 0)
qc1.x(0).c_if(creg, 1)

print(qc1)

statevect = qiskit_get_statevect(qc1)
print_state(statevect)

             ┌───┐                       ┌───┐                            »
q180_0: ─────┤ X ├─────────────────────■─┤ X ├───────■────────────────────»
             └─┬─┘┌───┐                │ └───┘       │                    »
q180_1: ───────┼──┤ X ├────────────────┼─────────────┼────■───────────────»
        ┌───┐  │  └─┬─┘          ┌───┐ │  ┌─┐      ┌─┴─┐┌─┴─┐┌───┐┌───┐┌─┐»
q180_2: ┤ H ├──■────■────■────■──┤ H ├─■──┤M├──|0>─┤ X ├┤ X ├┤ X ├┤ X ├┤M├»
        └───┘          ┌─┴─┐  │  └───┘    └╥┘      └───┘└───┘└─┬─┘└─┬─┘└╥┘»
q180_3: ───────────────┤ X ├──┼────────────╫───────────────────■────┼───╫─»
                       └───┘┌─┴─┐          ║                        │   ║ »
q180_4: ────────────────────┤ X ├──────────╫────────────────────────■───╫─»
                            └───┘          ║                            ║ »
  c1: 1/═══════════════════════════════════╩════════════════════════════╩═»
                                           0                            0 »
«         ┌─